# Visualise results from Optimisation of a year
This notebook shows the conducted analysis for the model results for each steel plant setup and its production with 2012 data for the whole year.  

In [ ]:
import flexible_batch_production as fbp
import numpy as np
import matplotlib.pyplot as plt

## Subsequent visualisation of one day or week in model period

First the model results, safed in the folder '1_output' need to be loaded with the corresponding method

In [ ]:
model_data = fbp.load_model('2012_flex_profit')

The time range is selected accordingly for one day

In [ ]:
start = 11232  # 19th March 2012 00:00
figure = fbp.analyse.loaded_model_summary_plot(model_data, np.datetime64('2012-01-01T00:00'), 
                                               time_range=range(start, start+144)) 

To analyse longer time series it is helpful to summarize model data into hourly chunks. The method hourlize_model_data does that by forming the mean over one hour chunks. 

In [ ]:
hour_model_data = fbp.hourlize_model_data(
    model_data, step_size=6, 
    do_not_hourlize = ['minutes_per_step', 'mass_production', 'mean_power_exchange']
)

start = 1872 # 19th March 2012 in hourly model
figure = fbp.loaded_model_summary_plot(
    hour_model_data, np.datetime64('2012-01-01T00:00'), 
    timedelta=np.timedelta64(60, 'm'),
    time_range=range(start, start+168)
                                        )

## Steel Plant Scenario Model Results Hourly Sorted
The following cell generates a hourly, annual summary plot as described in the thesis on this model.

In [ ]:
def what_to_sort(scenario): 
    if 'eafonly' in scenario:
      return ['minutes_per_step','mass_production','mean_power_exchange']
    else:
        return ['minutes_per_step','mass_production','mean_power_exchange', 'cost', 
                'power_from_grid', 'power_to_grid']    

scenarios = ['flex_profit', 'const_profit', 'eafonly_profit', 
             'flex_stability', 'const_stability', 'eafonly_stability']
titles = ['Demand Response Steel Making - Maximising Profits', 
          'Constant Steel Making - Maximising Profits', 
          'H2-DRI Import - Maximising Profits', 
          'Demand Response Steel Making - Minimise Goal Load Deviation', 
          'Constant Steel Making - Minimise Goal Load Deviation', 
          'H2-DRI Import - Minimise Goal Load Deviation'
          ]

for scenario, title in zip(scenarios, titles):
    model_data = fbp.load.load_model(f'2012_{scenario}')
    figure = fbp.analyse.sorted_model_summary_plot(
      fbp.hourlize_model_data(model_data), fill_line=0, alpha=1, title=title, 
      do_not_sort=what_to_sort(scenario)
    )
    figure.savefig(f'visualisations/hourlized/{scenario}', dpi=300)

## Further Model Analysis

For the further analysis of the model results several variables are examined in regards to input 
parameter and other variables at that time steps. As the total plant downtime is an important 
event to examine the following method is developed for analysis.

In [ ]:
def calc_pause_lengths(data):
    """ Calculates and returns the length of parts with continous value of 0 for given data and 
    the indexes for each step in a chunk"""
    zero_chunk_lengths = []
    zero_chunk_indexes = []

    i_pointer = 0
    while i_pointer < len(data):
        if data[i_pointer] == 0: # if a value is zero
            i_zero_chunk_start = i_pointer # safe index as start of possible zero chunk
            i_zero_chunk_end = i_pointer # declare index pointer to search for end of zero chunk
            while i_zero_chunk_end < len(data) and data[i_zero_chunk_end] == 0: # search for end of zero chunk
                i_zero_chunk_end += 1

            zero_chunk_length = len(data[i_zero_chunk_start:i_zero_chunk_end]) / 6  # 
            zero_chunk_lengths.append(zero_chunk_length)
            zero_chunk_indexes.append([i for i in range(i_zero_chunk_start, i_zero_chunk_end+1)])

            i_pointer = i_zero_chunk_end + 1
        else: # if there is not a zero value
            i_pointer += 1 # continue looking for zero chunks
    return zero_chunk_lengths, zero_chunk_indexes

### Wind generation during Plant Downtimes

In [ ]:
scenarios = ['flex_profit', 'const_profit', 'eafonly_profit', 
             'flex_stability', 'const_stability', 'eafonly_stability']
colors = ['#b0e3e6', '#ff7b59', '#edc86c', '#b0e3e6', '#ff7b59', '#edc86c']

figure, (axes_1, axes_2) = plt.subplots(nrows=2, ncols=3, sharex=False, figsize=(9, 6))
axes = np.append(axes_1, axes_2)

box_plot_liste = []

for scenario, ax, color in zip(scenarios, axes, colors): 
    model_data = fbp.load_model(f'2012_{scenario}')
    plant_load = [sum(x) for x in zip(model_data['electrolyser_load'], 
                                    model_data['steelmaking_load'], 
                                    model_data['rolling_load'])]
    downtime_length, downtime_indexes = calc_pause_lengths(plant_load)
    downtime_generation = [
        np.mean(model_data['renewable_generation'][downtime_i[0]:downtime_i[-1]]) 
        for downtime_i in downtime_indexes
    ]
    ax.scatter(downtime_generation, downtime_length, color=color, s=1)
    ax.annotate(f'n = {len(downtime_length)}', xy=(0.6, 0.8), xycoords='axes fraction')

axes[0].set_title('Demand Response Steel Making')
axes[1].set_title('Constant Steel Making')
axes[2].set_title('H2-DRI Import')

pad = 20 # in points
axes[0].annotate(
    'Maximise Profit', xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
    xycoords=axes[0].yaxis.label, textcoords='offset points', size='large', 
    ha='right', va='center'
)
axes[3].annotate(
    'Minimise Deviation\nof Goal Load', xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
    xycoords=axes[3].yaxis.label, textcoords='offset points', size='large', 
    ha='right', va='center'
)

figure.supylabel('Steel Plant Downtime Duration in hours', x=0.066, y=0.5, fontsize='large')
figure.supxlabel('Mean Renewable Power Generation during Downtime in MW', fontsize='large')
figure.savefig(f'visualisations/wind_in_downtime', bbox_inches='tight' , dpi=300)

### Electricity Prices during Plant Downtimes

In [ ]:
scenarios = ['flex_profit', 'const_profit', 'eafonly_profit', 
             'flex_stability', 'const_stability', 'eafonly_stability']
colors = ['#b0e3e6', '#ff7b59', '#edc86c', '#b0e3e6', '#ff7b59', '#edc86c']

figure, (axes_1, axes_2) = plt.subplots(nrows=2, ncols=3, sharex='col', figsize=(9, 6))
axes = np.append(axes_1, axes_2)

box_plot_liste = []

for scenario, ax, color in zip(scenarios, axes, colors): 
    model_data = fbp.load_model(f'2012_{scenario}')
    plant_load = [sum(x) for x in zip(model_data['electrolyser_load'], 
                                    model_data['steelmaking_load'], 
                                    model_data['rolling_load'])]
    downtime_length, downtime_indexes = calc_pause_lengths(plant_load)
    downtime_price = [
        np.mean(model_data['electricity_price'][downtime_i[0]:downtime_i[-1]]) 
        for downtime_i in downtime_indexes
    ]
    if scenario == 'eafonly_stability':
        ax = axes[5]
        index_to_del = downtime_price.index(-156.84857142857143)
        del downtime_price[index_to_del]
        del downtime_length[index_to_del]
    ax.scatter(downtime_price, downtime_length, color=color, s=1)

axes[0].set_title('Demand Response Steel Making')
axes[1].set_title('Constant Steel Making')
axes[2].set_title('H2-DRI Import')

pad = 20 # in points
axes[0].annotate(
    'Maximise Profit', xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
    xycoords=axes[0].yaxis.label, textcoords='offset points', size='large', 
    ha='right', va='center'
)
axes[3].annotate(
    'Minimise Deviation\nof Goal Load', xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
    xycoords=axes[3].yaxis.label, textcoords='offset points', size='large', 
    ha='right', va='center'
)

figure.supylabel('Steel Ploant Downtime Duration in hours', x=0.066, y=0.5,  fontsize='large')
figure.supxlabel('Mean Electricity Price during Downtime in €', fontsize='large')
figure.savefig("visualisations/price_in_downtime.png", bbox_inches='tight', dpi=300)

### Power Exchange with Electricity Grid related to Wind Generation and Prices

In [ ]:
scenarios = ['flex_profit', 'const_profit', 'eafonly_profit', 
             'flex_stability', 'const_stability', 'eafonly_stability']

figure, (axes_1, axes_2) = plt.subplots(nrows=2, ncols=3, figsize=(11, 6))
axes = np.append(axes_1, axes_2)

from matplotlib.colors import TwoSlopeNorm
norm = TwoSlopeNorm(vmin=-16, vcenter=42, vmax=100)

box_plot_liste = []

for scenario, ax, color in zip(scenarios, axes, colors): 
    model_data = fbp.load_model(f'2012_{scenario}')
    plot = ax.scatter(model_data['renewable_generation'], model_data['power_exchange'], 
                      c=model_data['electricity_price'], s=0.5, cmap='RdYlGn', norm=norm)

axes[0].set_title('Demand Response Steel Making')
axes[1].set_title('Constant Steel Making')
axes[2].set_title('H2-DRI Import')

pad = 20 # in points
axes[0].annotate(
    'Maximise Profit', xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
    xycoords=axes[0].yaxis.label, textcoords='offset points', size='large', 
    ha='right', va='center'
)
axes[3].annotate(
    'Minimise Deviation\nof Goal Load', xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
    xycoords=axes[3].yaxis.label, textcoords='offset points', size='large', 
    ha='right', va='center'
)
figure.subplots_adjust(right=0.8)
cbar_ax = figure.add_axes([0.85, 0.15, 0.05, 0.7])
ticks = ['< -16', '0', '20', '40', '60', '80', '> 100']
cbar = figure.colorbar(plot, label='Electricity Price in €/MWh',cax=cbar_ax)
cbar.ax.set_yticklabels(ticks)

figure.supxlabel('Renewable Generation in MW', fontsize='large')
figure.supylabel('Power Exchange between Plant and Grid in MW', x=0.067, y=0.5, fontsize='large')
figure.savefig("visualisations/wind_exchange_price.png", bbox_inches='tight', dpi=300)

### Monthly Mean Storage Contents and Steel Production

In [ ]:
scenarios = ['flex_profit', 'flex_stability','const_profit', 'const_stability', 'eafonly_profit', 'eafonly_stability']
scenario_labels = {
    'flex_profit': 'DR STM - Objective: Max Profit',
    'flex_stability': 'DR STM - Objective: Min Goal Load Deviation',
    'const_profit': 'Constant STM - Objective: Max Profit',
    'const_stability': 'Constant STM - Objective: Min Goal Load Deviation',
    'eafonly_profit': 'DRI Import - Objective: Max Profit',
    'eafonly_stability': 'DRI Import - Objective: Min Goal Load Deviation',
}
keys = ['DRI', 'Hydrogen Tank', 'Produced Steel']
key_units = {'DRI': 'tons', 'Hydrogen Tank': 'MWh', 'Produced Steel': 'tons'}
monthly_end_values = {}  
monthly_mean_values = {}

month_names = {1: 'Jan', 2: 'Feb', 3:'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 
               9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

for scenario in scenarios: 
    model_data = fbp.load.load_model(f'2012_{scenario}')
    
    monthly_end_values[scenario] = {key: {} for key in keys}
    monthly_mean_values[scenario] = {key: {} for key in keys if key != 'Produced Steel'}

    for i in range(1, 13):
        if i == 12 and (scenario == 'flex_stability' or scenario == 'const_stability'):
            monthly_end_values[scenario]['DRI'][month_names[i]] = model_data['DRI_storage_content'][i*4392-2]
            monthly_end_values[scenario]['Hydrogen Tank'][month_names[i]] = model_data['tank_hydrogen_content'][i*4392-2]
            monthly_end_values[scenario]['Produced Steel'][month_names[i]] = model_data['steel_produced'][i*4392-2] - model_data['steel_produced'][(i-1)*4392]

            monthly_mean_values[scenario]['DRI'][month_names[i]] = np.mean(model_data['DRI_storage_content'][(i-1)*4392:i*4392-2])
            monthly_mean_values[scenario]['Hydrogen Tank'][month_names[i]] = np.mean(model_data['tank_hydrogen_content'][(i-1)*4392:i*4392-2])

        else:
            monthly_end_values[scenario]['DRI'][month_names[i]] = model_data['DRI_storage_content'][i*4392-1]
            monthly_end_values[scenario]['Hydrogen Tank'][month_names[i]] = model_data['tank_hydrogen_content'][i*4392-1]
            monthly_end_values[scenario]['Produced Steel'][month_names[i]] = model_data['steel_produced'][i*4392-1] - model_data['steel_produced'][(i-1)*4392]

            monthly_mean_values[scenario]['DRI'][month_names[i]] = np.mean(model_data['DRI_storage_content'][(i-1)*4392:i*4392-1])
            monthly_mean_values[scenario]['Hydrogen Tank'][month_names[i]] = np.mean(model_data['tank_hydrogen_content'][(i-1)*4392:i*4392-1])
    

In [ ]:
figure, axes = plt.subplots(3, sharex=True, figsize=(9, 9))

for ax, key in zip(axes[:2], keys):
    for scenario in scenarios:
        ax.plot(monthly_mean_values[scenario][key].keys(), monthly_mean_values[scenario][key].values(), label=scenario_labels[scenario])
    ax.set_title(f'Mean Monthly {key} Storage Content')
    ax.set_ylabel(key_units[key])

for scenario in scenarios: 
    axes[2].plot(monthly_end_values[scenario]['Produced Steel'].keys(), 
                 monthly_end_values[scenario]['Produced Steel'].values(), 
                 label=scenario_labels[scenario])

axes[2].set_title('Produced Steel')
axes[2].set_ylabel('tons')

axes[1].legend()
figure.savefig("visualisations/storage_contents_andsteel_production.png", bbox_inches='tight', dpi=300)